<a href="https://colab.research.google.com/github/ileena/-ileena/blob/main/Build_Recommender_System_in_an_Hour___Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD

In [84]:
book = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, engine='python',encoding="latin-1")
book.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False,engine='python',encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False,engine='python',encoding="latin-1")
rating.columns = ['userID', 'ISBN', 'bookRating']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 815: ';' expected after '"'
Skipping line 1679: ';' expected after '"'
Skipping line 2531: ';' expected after '"'
Skipping line 2640: ';' expected after '"'
Skipping line 3236: ';' expected after '"'
Skipping line 3256: ';' expected after '"'
Skipping line 3733: unexpected end of data
Skipping line 269: ';' expected after '"'
Skipping line 1104: ';' expected after '"'
Skipping line 1274: ';' expected after '"'
Skipping line 1307: ';' expected after '"'
Skipping line 1696: ';' expected after '"'
Skipping line 1703: ';' expected after '"'
Skipping line 1999: ';' expected after '"'
Skipping line 2300: ';' expected after '"'
Skipping line 2450: ';' expected after '"'
Skipping line 2577: ';' expected after '"'
Skipping line 2665: ';' expected a

In [59]:
import pandas as pd
import numpy as np
import io
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD



book = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
book.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating.columns = ['userID', 'ISBN', 'bookRating']

In [60]:
rating.head()

,userID,ISBN,bookRating
0,276725,034545104X,0.000
1,276726,0155061224,5.000
2,276727,0446520802,0.000
3,276729,052165615X,3.000
4,276729,0521795028,6.000


In [61]:
user.head()

,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.000
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.000
4,5,"farnborough, hants, united kingdom",NaN


In [62]:
book.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [63]:
combine_book_rating = pd.merge(rating, book, on='ISBN')
columns = ['yearOfPublication', 'publisher', 'bookAuthor', 'imageUrlS', 'imageUrlM', 'imageUrlL']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

,userID,ISBN,bookRating,bookTitle
0,276725,034545104X,0.000,Flesh Tones: A Novel
1,2313,034545104X,5.000,Flesh Tones: A Novel
2,6543,034545104X,0.000,Flesh Tones: A Novel
3,8680,034545104X,5.000,Flesh Tones: A Novel
4,10314,034545104X,9.000,Flesh Tones: A Novel


### Filter to only popular books

Remove rows where book title is missing

In [64]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

In [65]:
book_ratingCount = (combine_book_rating.
     groupby(by = ['bookTitle'])['bookRating'].
     count().
     reset_index().
     rename(columns = {'bookRating': 'totalRatingCount'})
     [['bookTitle', 'totalRatingCount']]
    )
book_ratingCount.head()

,bookTitle,totalRatingCount
0,"100 Great Fantasy Short, Short Stories",1
1,101 Bright Ideas: Esl Activities for All Ages,1
2,101 Dalmatians,3
3,101 Telephone Jokes,1
4,11th Hour,1


#### Now we can merge the total rating count data into the rating data, giving us exactly what we need to filter out the lesser known books.

In [66]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'bookTitle', right_on = 'bookTitle', how = 'left')
rating_with_totalRatingCount.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount
0,276725,034545104X,0.000,Flesh Tones: A Novel,6
1,2313,034545104X,5.000,Flesh Tones: A Novel,6
2,6543,034545104X,0.000,Flesh Tones: A Novel,6
3,8680,034545104X,5.000,Flesh Tones: A Novel,6
4,10314,034545104X,9.000,Flesh Tones: A Novel,6


In [67]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   3608.000
mean       4.804
std        9.264
min        1.000
25%        1.000
50%        2.000
75%        5.000
max      241.000
Name: totalRatingCount, dtype: float64


#### The median book has only been rated one time. Let’s take a look at the top of the distribution.

In [68]:
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

0.900   11.000
0.910   12.000
0.920   14.000
0.930   15.000
0.940   16.580
0.950   19.000
0.960   22.000
0.970   26.000
0.980   32.000
0.990   43.930
Name: totalRatingCount, dtype: float64


#### So about 1% of books have 50 ratings, 2% have 29 ratings. Since we have so many books in our data, we’ll limit it to the top 1%, this will give us 2713 different books. 

In [69]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount
269,276925,0316666343,0.000,The Lovely Bones: A Novel,135
270,277195,0316666343,0.000,The Lovely Bones: A Novel,135
271,277413,0316666343,0.000,The Lovely Bones: A Novel,135
272,277427,0316666343,0.000,The Lovely Bones: A Novel,135
273,277439,0316666343,7.000,The Lovely Bones: A Novel,135


#### Filtering to US users only

In [70]:
combined = rating_popular_book.merge(user, left_on = 'userID', right_on = 'userID', how = 'left')

us_canada_user_rating = combined[combined['Location'].str.contains("usa|canada")]
us_canada_user_rating=us_canada_user_rating.drop('Age', axis=1)
us_canada_user_rating.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount,Location
1,277195,0316666343,0.000,The Lovely Bones: A Novel,135,"san francisco, california, usa"
2,277413,0316666343,0.000,The Lovely Bones: A Novel,135,"norfolk, virginia, usa"
3,277427,0316666343,0.000,The Lovely Bones: A Novel,135,"gilbert, arizona, usa"
4,277439,0316666343,7.000,The Lovely Bones: A Novel,135,"turner, oregon, usa"
5,277752,0316666343,7.000,The Lovely Bones: A Novel,135,"baton rouge, louisiana, usa"


In [71]:
if not us_canada_user_rating[us_canada_user_rating.duplicated(['userID', 'bookTitle'])].empty:
    initial_rows = us_canada_user_rating.shape[0]

    print('Initial dataframe shape {0}'.format(us_canada_user_rating.shape))
    us_canada_user_rating = us_canada_user_rating.drop_duplicates(['userID', 'bookTitle'])
    current_rows = us_canada_user_rating.shape[0]
    print('New dataframe shape {0}'.format(us_canada_user_rating.shape))
    print('Removed {0} rows'.format(initial_rows - current_rows))

Initial dataframe shape (1514, 6)
New dataframe shape (1503, 6)
Removed 11 rows


In [72]:
us_canada_user_rating_pivot = us_canada_user_rating.pivot(index = 'bookTitle', columns = 'userID', values = 'bookRating').fillna(0)
us_canada_user_rating_matrix = csr_matrix(us_canada_user_rating_pivot.values)

In [73]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(us_canada_user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [74]:
query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])

distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index, :].values.reshape(1,-1),n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for White Oleander : A Novel:

1: She's Come Undone (Oprah's Book Club), with distance of 0.7701844642859537:
2: Snow Falling on Cedars, with distance of 0.8486902897406724:
3: Summer Sisters, with distance of 0.8780694035036611:
4: The Bean Trees, with distance of 0.8873944035143412:
5: Life of Pi, with distance of 0.9060954612428757:


Perfect! "Green Mile Series" books are definitely should be recommended one after another.   

In [75]:
us_canada_user_rating_pivot2 = us_canada_user_rating.pivot(index = 'userID', columns = 'bookTitle', values = 'bookRating').fillna(0)

In [76]:
us_canada_user_rating_pivot2.head()

bookTitle,A Painted House,A Time to Kill,Angels &amp; Demons,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Good in Bed,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),Life of Pi,Little Altars Everywhere: A Novel,She's Come Undone (Oprah's Book Club),Snow Falling on Cedars,...,The Lovely Bones: A Novel,The Nanny Diaries: A Novel,The Pelican Brief,The Poisonwood Bible: A Novel,The Red Tent (Bestselling Backlist),The Secret Life of Bees,The Testament,Where the Heart Is (Oprah's Book Club (Paperback)),White Oleander : A Novel,Wild Animus
userID,,,,,,,,,,,,,,,,,,,,,
9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
14,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
67,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
114,0.000,0.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
165,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [77]:
us_canada_user_rating_pivot2.shape

(816, 26)

In [78]:
X = us_canada_user_rating_pivot2.values.T
X.shape

(26, 816)

In [79]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(26, 12)

In [80]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(26, 26)

In [88]:
us_canada_book_title = us_canada_user_rating_pivot2.columns
us_canada_book_list = list(us_canada_book_title)
White_Oleander = us_canada_book_list.index("She's Come Undone (Oprah's Book Club)")
print(White_Oleander)

8


In [90]:
corr_coffey_hands  = corr[White_Oleander]

In [ ]:
list(us_canada_book_title[(corr_coffey_hands<1.0) & (corr_coffey_hands>0.9)])

['Needful Things',
 'The Bachman Books: Rage, the Long Walk, Roadwork, the Running Man',
 'The Green Mile: Coffey on the Mile (Green Mile Series)',
 'The Green Mile: Night Journey (Green Mile Series)',
 'The Green Mile: The Bad Death of Eduard Delacroix (Green Mile Series)',
 'The Green Mile: The Mouse on the Mile (Green Mile Series)',
 'The Shining',
 'The Two Dead Girls (Green Mile Series)']

The results look great!